In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from statistics import mean
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import os
from torch.utils.data import DataLoader

In [2]:
base_path = '../../../../datasets/landscapes'
data_dir = datasets.ImageFolder(base_path)

train_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                       transforms.RandomRotation(30),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean= [0.485, 0.456, 0.406],
                                                            std= [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean= [0.485, 0.456, 0.406],
                                                            std= [0.229, 0.224, 0.225])])


train_data = datasets.ImageFolder(os.path.join(base_path, 'train'), transform=train_transforms)
test_data = datasets.ImageFolder(os.path.join(base_path, 'test'), transform=train_transforms)

trainloader = DataLoader(train_data, batch_size=64, shuffle=True)
testloader = DataLoader(test_data, batch_size=64, shuffle=False)

In [3]:
import time
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
model = models.googlenet(pretrained=True)

In [5]:
for param in model.parameters():
    param.requires_grad = False

from _collections import OrderedDict

classifier = nn.Sequential(OrderedDict([
    ('lin1', nn.Linear(1024, 256)),
    ('relu1', nn.ReLU()),
    # ('dp', nn.Dropout(p=0.2)),
    ('lin2', nn.Linear(256, 6)),
    ('log_softmax', nn.LogSoftmax(dim=1))]))

model.fc = classifier

In [6]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.0003)
model.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [7]:
print(model.conv1)

BasicConv2d(
  (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
)


In [8]:
print(model.inception5b.branch1)

BasicConv2d(
  (conv): Conv2d(832, 384, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn): BatchNorm2d(384, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
)


In [9]:
print(model.inception5b.branch1.conv)

Conv2d(832, 384, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [10]:
epochs = 10
print_every = 20

train = []
test = []

for epoch in range(epochs):
    steps = 0
    running_train_loss = 0
    running_test_loss = 0
    epoch_acc = []
    for inputs, labels in iter(trainloader):


        steps += 1

        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        train_pred = model.forward(inputs)

        loss = criterion(train_pred, labels)
        loss.backward()
        running_train_loss += loss.item()

        optimizer.step()



        if steps % print_every == 0:

            accuracy = 0
            model.eval()

            with torch.no_grad():
                for inputs, labels in iter(testloader):
                    inputs, labels = inputs.to(device), labels.to(device)

                    test_pred = model.forward(inputs)
                    test_loss = criterion(test_pred, labels)

                    running_test_loss += test_loss.item()

                    _, pred_class = test_pred.max(1)

                    correct = pred_class.eq(labels).sum().item()

                    accuracy = 100 * correct / len(testloader)

            test.append(running_test_loss/len(testloader))
            # load_acc = accuracy / len(testloader)

            model.train()

        train.append(running_train_loss/len(trainloader))

        # if steps % print_every == 0:
        #     print(f'Epoch: {epoch+1}/{epochs} | '
        #           f'Train loss: {running_train_loss/len(trainloader):.3f} | '
        #           f'Test loss: {running_test_loss/len(testloader):.3f} | '
        #           f'Accuracy: {accuracy:.3f}% |')

        running_train_loss = 0
        running_test_loss = 0

plt.plot(test)

C:\Users\lusty\anaconda3\envs\strive2\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Epoch: 1/10 | Train loss: 0.145 | Test loss: 1.333 | Accuracy: 89.286% |
Epoch: 1/10 | Train loss: 0.105 | Test loss: 0.949 | Accuracy: 76.786% |
Epoch: 1/10 | Train loss: 0.079 | Test loss: 0.718 | Accuracy: 85.714% |
Epoch: 1/10 | Train loss: 0.065 | Test loss: 0.621 | Accuracy: 87.500% |
Epoch: 1/10 | Train loss: 0.057 | Test loss: 0.553 | Accuracy: 92.857% |
Epoch: 1/10 | Train loss: 0.056 | Test loss: 0.531 | Accuracy: 89.286% |
Epoch: 1/10 | Train loss: 0.047 | Test loss: 0.489 | Accuracy: 94.643% |
Epoch: 1/10 | Train loss: 0.046 | Test loss: 0.462 | Accuracy: 91.071% |
Epoch: 1/10 | Train loss: 0.041 | Test loss: 0.441 | Accuracy: 87.500% |
Epoch: 1/10 | Train loss: 0.046 | Test loss: 0.442 | Accuracy: 89.286% |
Epoch: 1/10 | Train loss: 0.042 | Test loss: 0.427 | Accuracy: 78.571% |
Epoch: 2/10 | Train loss: 0.042 | Test loss: 0.437 | Accuracy: 85.714% |
Epoch: 2/10 | Train loss: 0.043 | Test loss: 0.398 | Accuracy: 89.286% |
Epoch: 2/10 | Train loss: 0.038 | Test loss: 0.407 

AttributeError: 'list' object has no attribute 'cpu'